# First steps for approaching the problem

## Pulse database

The task that we want the Neural Network to permorm is to *inverse map* the $N\times N$ real values that represent the SHG-FROG trace to the $2N$ real numbers that represent the real and imaginary parts of the electric field values.

<p align="center">
  <img src="../reports/figs/readme/NN.png" alt="NN scheme for solving the retrieval problem." width="550"/><br>
  <em>Figure 1: Scheme of the input and output layers of a Neural Network that solves the retrieval problem.</em>
</p>

Therefore, we will need to generate a database of simulated pulses containing both their SHG-FROG trace and the real and imaginary parts of the pulse electric field. An algorithm to perform such a task can be found at [this link](https://github.com/Loracio/ultrafast-pulse-retrieval/blob/main/src/pulse.hpp#L446) to the [`ultrafast-pulse-retrieval`](https://github.com/Loracio/ultrafast-pulse-retrieval) project, and can be used to create a database as required in this problem, as shown in the file [`testDataBaseGeneration.cpp`](https://github.com/Loracio/ultrafast-pulse-retrieval/blob/main/tests/testDataBaseGeneration.cpp).

The generated databases are .csv files containing the following structure:

|TBP | $E_{Re \ 0}$ | $\dots$ | $E_{Re \ N-1}$ | $E_{Im \ 0}$ | $\dots$ | $E_{Im \ N-1}$ | $T_{00}$ | $\dots$ | $T_{N-1 \ N-1}$

where:

- TBP: Time bandwidth product
- $E_{Re}$: Pulse real part (N columns)
- $E_{Im}$: Pulse imaginary part (N columns)
- $T_{mn}$: SHG-FROG trace of the pulse (NxN columns)

**Tensorflow** will be the library used in this project for the creation and training of Neural Networks, so the database has to be read and processed to be compatible with tensorflow data types.

A good practice is to normalize the input and the output of the Neural Network training and validation data. In this case, each trace is normalized by dividing by its maximum value, and the real and imaginary parts of each pulse are divided by the maximum amplitude (polar representation of complex numbers).

Database input/output handling functions are implemented inside the [`src/io`](/src/io/) folder, and will read and format the data into training and validation sets to train the Neural Network.

Now that we've got the data for training the Neural Network, we have to actually design and train it to solve our problem. We will use a *construction* approach, starting with the simplest possible model and gradually adding features that make it more complex and predictably better at solving the problem.

For a better handling and a faster training we will be using pulses with **N=64** for now on. This places limitations on the time-bandwidth product of the pulses we will be able to simulate, and therefore, on the ability of the network to produce pulses of a higher TBP.

## Multi Layer Perceptron as a first approach

The simplest architecture that can be used to solve the problem is a Multi Layer Perceptron (MLP), which is based on the basic unit of a neural network: the perceptron.

The architecture of this network will consist of an input layer with the NxN values of the SHG-FROG trace, one or more hidden layers of a variable number of neurons and an output with the 2N values of the electric field (real and imaginary parts).

There are several choices of hyperparameters that must be made to build the neural network:

- **Epochs**: number of epochs in which the network will be trained. To avoid overfitting, it is desirable to define an "early stopping" so that the training is stopped before the total number of epochs is reached if there is no improvement on the validation set.
- **Batch size**: number of training examples used in one iteration of the gradient descent. When the batch size is small, each update to the model parameters is based on fewer examples from the training set. This leads to more noise in the update process, which can help the model escape from local minima but also makes the training process less stable. On the other hand, when the batch size is large, each update is based on more examples, which makes the update process less noisy and more stable, but also more likely to get stuck in local minima. This is closer to the behavior of "batch" gradient descent, where each update is based on the entire training set.
- **Optimizer**: the optimizer in a neural network determines how the model updates its parameters in response to the calculated error gradient. The error gradient, which is computed using backpropagation, indicates the direction in which the model parameters need to be adjusted to minimize the loss function.
- **Learning rate**: the size of the steps taken to reach the minimum of the loss function. If the learning rate is too large, it may overshoot the optimal point. If it's too small, training will take too long.
- **Loss**: The function that the model tries to minimize during training. It's a measure of how far the model's predictions are from the true values.
- **Training and validation metrics**: metrics used to measure the performance of the model on the training and validation datasets.
- **Number of hidden layers**: number of layers in the neural network that are neither the input nor the output layer.
- **Number of neurons per hidden layer**: number of neurons in each hidden layer.
- **Activation function**: the function used to transform the input signal of a neuron in the network to its output signal.
- **Initializer**: the initializer of weights and biases in a neural network is a method or function that sets the initial values of the model's parameters (weights and biases) before training starts. The choice of initialization can significantly affect the speed of convergence and the final performance of the model.
- **Dropout**: regularization technique where randomly selected neurons are ignored during training. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass. It helps prevent overfitting.

Deciding which of these parameters to use often comes down to a process of trial and error, although it is possible to choose them with "a bit of common sense".

Let's get started with the coding. To record the neural network trainings we will use the [Weights & Biases (wandb)](https://wandb.ai/site) tool. This will allow us to have in a clean and orderly way each of the networks that we are going to train in different projects, where we will be able to see the different graphs of metrics and network hyperparameters used.

If you want to use the script version of this notebook, check [`/tests/test_launch_MLP.py`](/tests/test_launch_MLP.py).

In [1]:
# We start with importing the modules we'll need
import numpy as np
import tensorflow as tf
from tensorflow import keras


import wandb # Weights and Biases
from wandb.keras import WandbCallback # We'll use this to log our metrics to Weights and Biases

2024-02-09 00:54:24.167573: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-09 00:54:24.190776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


As mentioned above, we will use data with N=64 samples. We generate a database containing 2500 pulses and create a variable pointing to its path.

In [2]:
N = 64
NUMBER_OF_PULSES = 1000
FILE_PATH = f"../data/generated/N{N}/{NUMBER_OF_PULSES}_randomPulses_N{N}.csv"
# Handle error if path does not exist
try:
    with open(FILE_PATH) as f:
        pass
except FileNotFoundError:
    print("File not found. Please generate the pulse database first.")
    exit()

Wandb uses a dictionary to store the model's parameters. We can store whatever we want and think its relevant to reproduce the model.

In [3]:
config = {
    'epochs': 50,
    'batch_size': 256,
    'log_step': 50,
    'val_log_step': 50,
    'optimizer': 'adam',
    'learning_rate': 0.001,
    'loss': 'mse',
    'train_metrics': 'MeanSquaredError',
    'val_metrics': 'MeanSquaredError',
    'n_hidden_layers': 2,
    'n_neurons_per_layer': 512,
    'activation': 'relu',
    'dropout': 0.1,
    'patience': 10,
    'training_size': 0.8,
    'database': f'{NUMBER_OF_PULSES}_randomPulses_N{N}',
    'arquitecture': 'MLP',
    'input_shape': (N, N),
    'output_shape': (int(2 * N)),
}

We define the function `load_and_norm_data` that returns a `tf.data.Dataset` object with the normalized pulse database read from file. All traces are normalized by their maximum value. The real part and imaginary part of the pulse are normalized by dividing by the maximum absolute value (module) of the complex number. Note that we also have the TBP of the pulses in the first column of the db. 

Then, defining the function `process_data` the dataset is processed. Data is batched and shuffled, dividing it into train and test sets. The output are two datasets containing the two sets.

In [4]:
def load_and_norm_data(FILE_PATH, N, NUMBER_OF_PULSES):
    """
    This function preprocesses the data from the database, iterating over it.
    It returns a tensorflow dataset.

    Note that we also have the TBP of the pulses in the first column of the db.
    We want to save them in a separate array, so we can use them later.

    The function also normalizes each trace, by dividing by the maximum value.
    The real part and imaginary part of the pulse are normalized by dividing by
    the maximum absolute value (module) of the complex number.

    Args:
        FILE_PATH: str
            Path to the database file
        N: int
            Number of points in the SHG-FROG trace
        NUMBER_OF_PULSES: int
            Number of pulses in the database

    Returns:
        dataset: tf.data.Dataset
            Dataset with the pulse database
    """
    # Create a record_defaults with 1 + 2N + N*N elements that are tf.float32
    db_record_defaults = [tf.float32] * (1 + 2*N + N*N)

    # Read the database
    pulse_db = tf.data.experimental.CsvDataset(
        FILE_PATH, record_defaults=db_record_defaults, header=False)

    # Create empty datasets
    tbp_dataset = tf.data.Dataset.from_tensor_slices([])
    train_dataset = tf.data.Dataset.from_tensor_slices([])
    target_dataset = tf.data.Dataset.from_tensor_slices([])

    # Iterate over the database
    for i, pulse in enumerate(pulse_db):
        # Save the TBP in the tbp_dataset
        tbp_dataset = tbp_dataset.concatenate(
            tf.data.Dataset.from_tensor_slices(tf.reshape(pulse[0], (1,))))

        # Save the SHG-FROG trace in the train_dataset and normalize
        shg_frog_trace = tf.reshape(pulse[2*N + 1:], (1, N, N))
        normalized_trace = shg_frog_trace / tf.reduce_max(tf.abs(shg_frog_trace))
        train_dataset = train_dataset.concatenate(
            tf.data.Dataset.from_tensor_slices(normalized_trace))

        # Save the pulse in the target_dataset and normalize
        pulse_real = tf.reshape(pulse[1:N + 1], (1, N))
        pulse_imag = tf.reshape(pulse[N + 1:2*N + 1], (1, N))

        # Combine real and imaginary parts into complex numbers
        pulse_complex = tf.complex(pulse_real, pulse_imag)

        # Find the maximum absolute value (module) of the complex numbers
        max_module = tf.reduce_max(tf.abs(pulse_complex))

        # Normalize the real and imaginary parts by the maximum module
        normalize_pulse_real = pulse_real / max_module
        normalize_pulse_imag = pulse_imag / max_module

        normalized_pulse = tf.concat([normalize_pulse_real, normalize_pulse_imag], axis=1)
        target_dataset = target_dataset.concatenate(
            tf.data.Dataset.from_tensor_slices(normalized_pulse))

    # Create the final dataset
    dataset = tf.data.Dataset.zip((tbp_dataset, train_dataset, target_dataset))

    return dataset

In [5]:
def process_data(N, NUMBER_OF_PULSES, pulse_dataset, training_size, BATCH_SIZE, SHUFFLE_BUFFER_SIZE=None):
    """
    In this function the data is processed.
    Data is batched and shuffled, dividing it into train and test sets.

    Args:
        N (int): Number of time steps
        NUMBER_OF_PULSES (int): Number of pulses in the database
        pulse_dataset (tf.data.Dataset): Dataset containing the pulses

    Returns:
        train_dataset (tf.data.Dataset): Dataset containing the training pulses
        test_dataset (tf.data.Dataset): Dataset containing the test pulses
    """
    if SHUFFLE_BUFFER_SIZE is None:
        SHUFFLE_BUFFER_SIZE = NUMBER_OF_PULSES

    # Select the y and z data from pulse dataset, which contain the SHG-FROG trace and the electric field of the pulse in the time domain
    pulse_dataset = pulse_dataset.map(lambda x, y, z: (y, z))

    # Split the dataset into train and test, shuffle and batch the train dataset
    train_dataset = pulse_dataset.take(int(training_size * NUMBER_OF_PULSES)).shuffle(buffer_size=SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
    test_dataset = pulse_dataset.skip(int(training_size * NUMBER_OF_PULSES)).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_dataset, test_dataset

In [6]:
# Call the functions
pulse_dataset = load_and_norm_data(FILE_PATH, N, NUMBER_OF_PULSES)
train_dataset, test_dataset = process_data(N, NUMBER_OF_PULSES, pulse_dataset,
                                            config['training_size'], config['batch_size'])

In [7]:
# Initialize wandb for the tracking. 
# We need to pass the configuration, the project name and name of the run
run = wandb.init(project="MLP_example", config=config,
                    name='MLP test run #1')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vloras (pulse-retrieval-with-nn). Use `wandb login --relogin` to force relogin


We define a function that creates a Multi Layer Perceptron based on the values of the config dictionary.

In [8]:
def MLP(input_shape, output_shape, n_hidden_layers, n_neurons_per_layer, activation, dropout=None):
    """
    This function creates a MLP model with the specified parameters.

    Args:
        input_shape (tuple): Shape of the input layer
        output_shape (tuple): Shape of the output layer
        n_hidden_layers (int): Number of hidden layers
        n_neurons_per_layer (int): Number of neurons per hidden layer
        activation (str): Activation function to use in hidden layers
    """

    inputs = keras.Input(shape=input_shape, name="input")
    flatten_layer = keras.layers.Flatten()(inputs)
    # Add the hidden layers given by the arguments
    dense_layer = keras.layers.Dense(n_neurons_per_layer, activation=activation)(flatten_layer)
    # Only add dropout if dropout is not None
    if dropout is not None:
            dense_layer = keras.layers.Dropout(dropout)(dense_layer)
    for i in range(n_hidden_layers - 1):
        dense_layer = keras.layers.Dense(n_neurons_per_layer, activation=activation)(dense_layer)
        # Only add dropout if dropout is not None
        if dropout is not None:
            dense_layer = keras.layers.Dropout(dropout)(dense_layer)
    outputs = keras.layers.Dense(output_shape, name="output")(dense_layer)

    return keras.Model(inputs=inputs, outputs=outputs)

In [9]:
# Build the model with the config
model = MLP(config['input_shape'], config['output_shape'], config['n_hidden_layers'],
            config['n_neurons_per_layer'], config['activation'], config['dropout'])

# Print the model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 64, 64)]          0         
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 512)               2097664   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 output (Dense)              (None, 128)               65664 

We have now to define the training step of the NN. It will consist of a training step with the training dataset and a validation step with the validation dataset. Additionally, the metrics will be stored with wandb after each epoch.

In [10]:
def train_step_MLP(x, y, model, optimizer, loss_fn, train_acc_metric):
    """
    Example training step for a MLP model.

    Args:
        x (tf.Tensor): Input data
        y (tf.Tensor): Target data
        model (tf.keras.Model): Model to train
        optimizer (tf.keras.optimizers.Optimizer): Optimizer to use
        loss_fn (tf.keras.losses.Loss): Loss function to use
        train_acc_metric (tf.keras.metrics.Metric): Metric to use for training accuracy

    Returns:
        loss_value (tf.Tensor): Loss value for the training step
    """
    with tf.GradientTape() as tape:
        results = model(x, training=True)
        loss_value = loss_fn(y, results)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, results)

In [11]:
def test_step_MLP(x, y, model, loss_fn, test_acc_metric):
    """
    Example test step for a MLP model.

    Args:
        x (tf.Tensor): Input data
        y (tf.Tensor): Target data
        model (tf.keras.Model): Model to train
        loss_fn (tf.keras.losses.Loss): Loss function to use
        test_acc_metric (tf.keras.metrics.Metric): Metric to use for test accuracy

    Returns:
        loss_value (tf.Tensor): Loss value for the validation step
    """
    val_results = model(x, training=False)
    loss_value = loss_fn(y, val_results)

    test_acc_metric.update_state(y, val_results)

    return loss_value

In [12]:
def train_MLP(train_dataset, test_dataset, model, optimizer, loss_fn, train_acc_metric, test_acc_metric, epochs, log_step, val_log_step, patience):
    """
    Trainin step for a MLP model. Updates the weights of the model using the gradients computed by the loss function.
    Saves the training and validation loss and accuracy in wandb.

    Args:
        train_dataset (tf.data.Dataset): Training dataset.
        test_dataset (tf.data.Dataset): Test dataset.
        model (tf.keras.Model): Model to train.
        optimizer (tf.keras.optimizers): Optimizer to use.
        loss_fn (tf.keras.losses): Loss function to use.
        train_acc_metric (tf.keras.metrics): Training accuracy metric.
        test_acc_metric (tf.keras.metrics): Test accuracy metric.
        epochs (int): Number of epochs to train.
        log_step (int): Number of steps to log training metrics.
        val_log_step (int): Number of steps to log validation metrics.
        patience (int): Number of epochs to wait for improvement in validation loss before early stopping.
    """
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            loss_value = train_step_MLP(x_batch_train, y_batch_train,
                                    model, optimizer,
                                    loss_fn, train_acc_metric)
            average_loss_value = tf.reduce_mean(loss_value)
            train_loss.append(float(average_loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(test_dataset):
            val_loss_value = test_step_MLP(x_batch_val, y_batch_val,
                                       model, loss_fn,
                                       test_acc_metric)
            average_loss_value = tf.reduce_mean(val_loss_value)
            val_loss.append(float(average_loss_value))

        avg_val_loss = np.mean(val_loss)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter > patience:
            print("Early stopping due to no improvement in validation loss")
            break

        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        test_acc = test_acc_metric.result()
        print("Validation acc: %.4f" % (float(test_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_states()
        test_acc_metric.reset_states()


        # log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'train_loss': np.mean(train_loss),
                   'train_acc': float(train_acc),
                   'test_loss': np.mean(val_loss),
                   'test_acc': float(test_acc)
                   })

We now have to initialize the optimizer with the selected learning rate and call the `train_MLP` function to train the model.

In [13]:
# Set the optimizer with the config with its learning rate
optimizer = keras.optimizers.get(config['optimizer'])
optimizer.learning_rate = config['learning_rate']


# Train the model with the config
train_MLP(train_dataset, test_dataset, model,
            epochs=config['epochs'],
            optimizer=optimizer,
            loss_fn=keras.losses.get(config['loss']),
            train_acc_metric=keras.metrics.get(config['train_metrics']),
            test_acc_metric=keras.metrics.get(config['val_metrics']),
            log_step=config['log_step'],
            val_log_step=config['val_log_step'],
            patience=config['patience']
            )

# Finish the run in wandb
run.finish()

# Save the model
model.save(f"./trained_models/FCNN/{config['arquitecture']}_test.h5")


Start of epoch 0


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

I don't know why, but this doesn't work here on the Jupyter Notebook but it works if used in a normal Python script.

Anyways, now the results of the training can be viewed on the Weights & Biases webpage.

For taking a look at the results of the network, we can use the GUI class defined in [`/src/visualization/visualization.py`](/src/visualization/visualization.py) to view the results.

Check an example in [`/tests/test_visualization.py`](/tests/test_visualization.py).